# Ship or Iceberg
https://www.kaggle.com/c/statoil-iceberg-classifier-challenge

In [25]:
import torch
import torch.autograd as autograd
import torch.nn as nn

import numpy as np
import pandas as pd

# Data

In [2]:
datadir = "/home/christopher/Data/data/ml/ship-or-iceberg/"

with open(datadir + "train.json") as f:
    train = pd.read_json(f).set_index("id")
train.head()

,band_1,band_2,inc_angle,is_iceberg
id,,,,
dfd5f913,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",43.9239,0
e25388fd,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",38.1562,0
58b2aaa0,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",45.2859,1
4cfc3a18,"[-22.454607, -23.082819, -23.998013, -23.99805...","[-27.889421, -27.519794, -27.165262, -29.10350...",43.8306,0
271f93f4,"[-26.006956, -23.164886, -23.164886, -26.89116...","[-27.206915, -30.259186, -30.259186, -23.16495...",35.6256,0


In [40]:
band1 = np.array([np.array([i]) for i in train["band_1"]], dtype=np.float32)
band1 = band1.reshape(len(band1), 1, 75, 75)
band2 = np.array([np.array([i]) for i in train["band_2"]], dtype=np.float32)
band2 = band2.reshape(len(band2), 1, 75, 75)

train_data = np.concatenate((band1, band2), axis=1)
labels = np.array(train["is_iceberg"])

In [41]:
print(train_data.shape)
print(labels.shape)

(1604, 2, 75, 75)
(1604,)


# Neural Network

In [53]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(2, 3, 4) # in_chan, out_chan, kernel_size
        self.pool = nn.MaxPool2d(3, 3)
        self.lin1 = nn.Linear(3 * 24 * 24, 100) # in_feat, out_feat
        self.lin2 = nn.Linear(100, 2)
    
    def forward(self, x):
        print(x.shape)
        x = self.conv1(x)
        print(x.shape)
        x = self.pool(x)
        print(x.shape)
        x = x.view(3*24*24)
        print(x.shape)
        # 2x75x75 -> 3x72x72 -> 3x24x24
        x = self.lin1(x)
        print(x.shape)
        x = self.lin2(x)
        print(x.shape)
        return x
net = Net()

In [54]:
print(type(train_data[0][0][0][0]))
inp = autograd.Variable(torch.from_numpy(train_data[0:1]))
print(inp)
net(inp)

<class 'numpy.float32'>
Variable containing:
(0 ,0 ,.,.) = 
 -27.8784 -27.1542 -28.6686  ...  -25.5735 -26.4887 -30.5070
 -28.6685 -27.8784 -27.5088  ...  -26.4886 -28.6716 -31.5942
 -28.6685 -27.1542 -25.8650  ...  -27.1571 -27.8814 -32.8371
            ...               ⋱              ...            
 -29.0929 -28.6692 -30.5046  ...  -30.0110 -25.8685 -26.4893
 -31.5917 -27.8790 -28.6692  ...  -27.5123 -26.8171 -27.5124
 -29.0929 -26.8141 -27.1548  ...  -26.8170 -27.8820 -28.2681

(0 ,1 ,.,.) = 
 -27.1541 -29.5379 -31.0306  ...  -29.0953 -29.5409 -27.5118
 -31.0305 -33.5293 -31.5912  ...  -34.2881 -28.2675 -26.4887
 -33.5293 -37.0512 -31.5912  ...  -31.5941 -32.1934 -29.5409
            ...               ⋱              ...            
 -27.8790 -27.8790 -30.0081  ...  -25.2892 -27.5123 -28.6722
 -29.5384 -29.5385 -29.0930  ...  -28.6721 -28.2681 -33.5330
 -30.0081 -28.6692 -27.5094  ...  -30.5075 -28.6722 -29.0960
[torch.FloatTensor of size 1x2x75x75]

torch.Size([1, 2, 75, 75])
torc

Variable containing:
-1.3606
 5.1510
[torch.FloatTensor of size 2]